# *Bacillus subtilis* Diauxic Shift

Here we will attempt to reproduce the results from [Rosenthal et al. 2018](https://elifesciences.org/articles/33099). 

The shared environment will contain 50 mM Malate and 22 mM Glucose. We will assume that the environment is 1L in volume - to avoid unit conversions. 

Initial biomass is set to 0.5.

In [3]:
from copy import deepcopy

from process_bigraph import Composite
from process_bigraph.composite import ProcessTypes

from cdFBA import register_types
from cdFBA.processes.dfba import DFBA, UpdateEnvironment, StaticConcentration, Injector, WaveFunction

from cdFBA.utils import make_cdfba_composite, get_injector_spec, get_wave_spec, get_static_spec, model_from_file
from cdFBA.utils import get_exchanges, get_substrates, get_reaction_map

from matplotlib import pyplot as plt
from pprint import pprint

import cProfile
import pstats
import io

<unknown>:1: SyntaxWarning: invalid escape sequence '\s'


In [4]:
model_file = 'Bacillus_subtilis_str_168.xml'

In [ ]:
model_dict = {'B. subtilis': model_file}
exchanges = ['EX_ac(e)','EX_actn_R(e)', 'EX_glc_D(e)']
#'EX_mal_L(e)', 'EX_glc_D(e)'
volume = 1
spec = make_cdfba_composite(model_dict, medium_type=None, exchanges=exchanges, volume=volume, interval=0.5)

In [ ]:
#Set reaction bounds
spec['B. subtilis']['config']['bounds'] = {
            'EX_o2(e)': {'lower': -2, 'upper': None},
            'DM_atp_c_': {'lower': 1, 'upper': 1}
        }
#Set initial concentrations
spec['shared environment']['concentrations']['Acetate'] = 0
spec['shared environment']['concentrations']['D-Glucose'] = 2.2
# spec['shared environment']['concentrations']['L-malate'] = 5.5
spec['shared environment']['concentrations']['R Acetoin'] = 0
spec['shared environment']['counts']['Acetate'] = 0/volume
spec['shared environment']['counts']['D-Glucose'] = 2.2/volume
# spec['shared environment']['counts']['L-malate'] = 5.5/volume
spec['shared environment']['counts']['R Acetoin'] = 0/volume

In [ ]:
#set emitter specs
spec['emitter'] = {
        "_type": "step",
        "address": "local:ram-emitter",
        "config": {
            "emit": {
                "shared_environment": "any",
                "global_time": "any",
            }
        },
        "inputs": {
            "shared_environment": ["shared environment"],
            "global_time": ["global_time"]
        }
    }

In [ ]:
#create the core object
core = ProcessTypes()
#register data types
core = register_types(core)
#register all processes and steps
core.register_process('DFBA', DFBA)
core.register_process('UpdateEnvironment', UpdateEnvironment)
core.register_process('StaticConcentration', StaticConcentration)
core.register_process('WaveFunction', WaveFunction)
core.register_process('Injector', Injector)

In [ ]:
#create simulation composite
sim = Composite({
        "state": spec,
        },
        core=core
    )

In [ ]:
sim.run(10)

In [ ]:
results = sim.gather_results()[('emitter',)]

In [ ]:
#extract time-series data
timepoints = []
for timepoint in results:
    time = timepoint.pop('global_time')
    timepoints.append(time)
env = [timepoint['shared_environment']['concentrations'] for timepoint in results]
env_combined = {}
for d in env:
    for key, value in d.items():
        if key not in env_combined:
            env_combined[key] = []
        env_combined[key].append(value)

In [ ]:
#plot results for biomass
fig, ax = plt.subplots(dpi=100)
for key, value in env_combined.items():
    if key not in []:
        ax.plot(timepoints, env_combined[key], label=key)
plt.xlabel('Time')
plt.ylabel('Substrate Concentration')
plt.legend()
plt.tight_layout()
plt.show()
#'B. subtilis', 'Acetate', 'R Acetoin'

In [ ]:
results

In [5]:
model = model_from_file(model_file)

In [6]:
results = model.optimize()

In [10]:
results.fluxes['biomass166']

np.float64(112.66608490628785)

In [8]:
from cdFBA.utils import get_objective_reaction

In [9]:
get_objective_reaction(model_file=model_file)

'biomass166'

In [19]:
model = model_from_file(model_file)

In [20]:
getattr(model.reactions, 'EX_mal_L(e)').bounds

(-1000.0, 1000.0)

In [21]:
fluxes = model.optimize().fluxes

In [29]:
intake_fluxes = fluxes[fluxes < 0]

In [37]:
intake_reactions = [getattr(model.reactions, i).name for i in list(intake_fluxes.index)]

In [32]:
len(intake_fluxes)

165

In [34]:
exchanges = [reaction.name for reaction in model.exchanges]

In [40]:
sorted([reaction for reaction in intake_reactions if reaction in exchanges])

['(R)-Pantothenate exchange',
 '2-Oxobutanoate exchange',
 '4 Aminobenzoate exchange',
 '4-Aminobutanoate exchange',
 '4-Hydroxybenzoate exchange',
 'Adenosylcobalamin exchange',
 'Balsalazide exchange',
 'Calcium exchange',
 'Chenodeoxyglycocholate exchange',
 'Citrate exchange',
 'Co2+ exchange',
 'Cu2+ exchange',
 'Cys-Gly exchange',
 'Exchange of 3-methyl-2-oxopentanoate',
 'Exchange of Glycerophosphoric Acid',
 'Fe3+ exchange',
 'Glycocholate exchange',
 'Glycyl-L-asparagine exchange',
 'Glycyl-L-tyrosine exchange',
 'Glycylleucine exchange',
 'H2O exchange',
 'Hydrogen sulfide exchange',
 'K+ exchange',
 'L-Arginine exchange',
 'L-Aspartate exchange',
 'L-Lysine exchange',
 'L-Proline exchange',
 'L-Tryptophan exchange',
 'L-Valine exchange',
 'L-alanyl-L-threonine exchange',
 'L-methionyl-L-alanine exchange',
 'Lactulose exchange',
 'Maltohexaose exchange',
 'Menaquinone 8 exchange',
 'Mg exchange',
 'Mn2+ exchange',
 'N-Acetylisoniazid exchange',
 'Nicotinate exchange',
 'Olsal